## Pacotes necessários

In [52]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Lendo o banco de dados

In [53]:
dados = pd.read_csv('./train.csv')
dados.head()

dados = dados.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
dados.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


## Editando **chave** e **variáveis resposta**

In [54]:
dados = dados.set_index(['PassengerId'])
dados = dados.rename(columns={'Survived': 'target'}, inplace=False)

dados.head()

,target,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
1,0,3,male,22.0,1,0,7.2500
2,1,1,female,38.0,1,0,71.2833
3,1,3,female,26.0,0,0,7.9250
4,1,1,female,35.0,1,0,53.1000
5,0,3,male,35.0,0,0,8.0500


## Descritiva

In [55]:
# count = contagem
# mean = média (40% são sobreviventes)
# std = desvio

dados.describe()

,target,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [56]:
# top = sexo mais frequente => male (masculino)
# freq = quantos ocorrências do sexo masculino => 577

dados.describe(include=['O'])

,Sex
count,891
unique,2
top,male
freq,577


## Transformação dos dados

In [57]:
# Sexo feminino = 1 | Sexo masculino = 0 (flag)
dados['Sex_F'] = np.where(dados['Sex'] == 'female', 1, 0)

# Classe do passageiro: se for "Classe 1" => 1, se não => 0
dados['Pclass_1'] = np.where(dados['Pclass'] == 1, 1, 0)

# Classe do passageiro: se for "Classe 2" => 1, se não => 0
dados['Pclass_2'] = np.where(dados['Pclass'] == 2, 1, 0)

# Classe do passageiro: se for "Classe 3" => 1, se não => 0
dados['Pclass_3'] = np.where(dados['Pclass'] == 3, 1, 0)

In [58]:
# Remoção das variáveis "crú"
dados = dados.drop(['Pclass', 'Sex'], axis=1)

In [59]:
dados.head()

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1
2,1,38.0,1,0,71.2833,1,1,0,0
3,1,26.0,0,0,7.9250,1,0,0,1
4,1,35.0,1,0,53.1000,1,1,0,0
5,0,35.0,0,0,8.0500,0,0,0,1


In [60]:
# Quantidade de dados nulos
dados.isnull().sum()

# Conclusão: Apenas a "idade" possui dados nulos

target        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex_F         0
Pclass_1      0
Pclass_2      0
Pclass_3      0
dtype: int64

In [61]:
# Substituição de "missings" (dados nulos)
dados.fillna(0, inplace=True)

In [62]:
# Garantindo que não tem mais nenhum dado nulo
dados.isnull().sum()

target      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex_F       0
Pclass_1    0
Pclass_2    0
Pclass_3    0
dtype: int64

## Amostragem

In [63]:
# Algoritmo de amostragem
x_train, x_test, y_train, y_test = train_test_split(
    dados.drop(['target'], axis=1),
    dados['target'],
    test_size=0.3,
    random_state=1234
)

# Base de dados:
# 70% => treinar o modelo
# 30% => testar o modelo
# Essa separação serve para o modelo não ficar específico essa base de dados
[{'treino': x_train.shape}, {'teste': x_test.shape}]

[{'treino': (623, 8)}, {'teste': (268, 8)}]

## Modelo (**Random Forest**)

In [64]:
# "n_estimators" = quantidade de árvores à serem feitas
# "criterion" = critério de criação
# "max_depth" = profundidade máximo (em nós)
rndforest = RandomForestClassifier(
    n_estimators=1000,
    criterion='gini',
    max_depth=5
)

rndforest.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000)

In [65]:
# "probabilidade" de sobrevivência = variação de 0 a 1
probabilidade = rndforest.predict_proba(
    dados.drop('target', axis=1)
)[:,1]

# "classificacao" = "probabilidade" < 0.5 => não sobreviveu (0) | "probabilidade" >= 0.5 => sobreviveu (1)
classificacao = rndforest.predict(
    dados.drop('target', axis=1)
)

In [66]:
dados['probabilidade'] = probabilidade
dados['classificacao'] = classificacao

In [67]:
dados

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3,probabilidade,classificacao
PassengerId,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1,0.123255,0
2,1,38.0,1,0,71.2833,1,1,0,0,0.944509,1
3,1,26.0,0,0,7.9250,1,0,0,1,0.440279,0
4,1,35.0,1,0,53.1000,1,1,0,0,0.922930,1
5,0,35.0,0,0,8.0500,0,0,0,1,0.138382,0
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.0,0,0,13.0000,0,0,1,0,0.129153,0
888,1,19.0,0,0,30.0000,1,1,0,0,0.859216,1
889,0,0.0,1,2,23.4500,1,0,0,1,0.492323,0
